In [1]:
"""
Preprocess simulation data from Claudio so we can train NN on it.
Author Peter
Updated Aug 29, 2023 by Linnea

Transformations:
1) Transpose flux to make it easier to use dataloaders. NOW DONE IN PREPREPROCESSING STEP.
2) Remove elements that have bad quality flag.
3) Permute data for training. 
"""

import h5py
import numpy as np

import preprocess
%load_ext autoreload
%autoreload 2

path_shared = '/home/linneamw/sadow_lts/shared/gcr/data/2023_07_01'
path_personal = '/home/linneamw/sadow_lts/personal/linneamw/research/gcr/data/2023_07_01'

In [2]:
# Positive polarity
origfile = f'{path_shared}/pos/model_collection_1AU_90deg_0deg.h5'
infile = f'{path_personal}/pos/model_collection_1AU_90deg_0deg_fixed.h5'
outfile = f'{path_personal}/pos/model_collection_1AU_90deg_0deg_fixed_training.h5'
preprocess.prepreprocess(infile=origfile, outfile=infile)
preprocess.make_preprocessed_file(infile, outfile)

['info', 'model']
['LIS', 'rigidity']
['alpha', 'cmf', 'cpa', 'flux', 'imodel', 'ipar', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'quality', 'vseq', 'vspoles']
Found samples: 2851200
Num flux targets: 32


In [3]:
# Negative polarity
origfile = f'{path_shared}/neg/model_collection_1AU_90deg_0deg.h5'
infile = f'{path_personal}/neg/model_collection_1AU_90deg_0deg_fixed.h5'
outfile = f'{path_personal}/neg/model_collection_1AU_90deg_0deg_fixed_training.h5'
preprocess.prepreprocess(infile=origfile, outfile=infile)
preprocess.make_preprocessed_file(infile, outfile)

['info', 'model']
['LIS', 'rigidity']
['alpha', 'cmf', 'cpa', 'flux', 'imodel', 'ipar', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'quality', 'vseq', 'vspoles']
Found samples: 2851200
Num flux targets: 32
